In [1]:
# This notebook summarizes the simulation construction and running for net 01

In [2]:
# Defining network file
NET_FILE = ".\data/grid01-bcn_06.net.xml"

In [3]:
# Demand generation
# Endogenous demand based on ACTIVITYGEN

## Config file for parsing demographic data:

ACTIVITYGEN_STAT_FILE = ".\data/activitygen-bcn_06.stat.xml"

## Basic trip output file (without routing) as a resukt of the demand generation based on ACTIVITYGEN.
TRIPS_FROM_ACTIVITYGEN_STAT = ".\data/activitygen-bcn_06.trips.rou.xml"

In [ ]:
# Calling ACTIVITYGEN tool in SUMO
# Random seed set for reproductibility purposes

!activitygen --net-file {NET_FILE} \
            --stat-file {ACTIVITYGEN_STAT_FILE} \
            --output-file {TRIPS_FROM_ACTIVITYGEN_STAT} \
            --seed 25101987